In [ ]:
from methods.cluster_topo import inside_topo,surf_topo
from ase.io import read
filename = './examples/tetrahedron.vasp'
atoms = read(filename)
inside_sites = inside_topo(atoms,bond_len=1.6)
print("inside_sites:",len(inside_sites))
surf_sites = surf_topo(atoms,bond_len=1.6)
print("surf_sites:",len(surf_sites))

In [9]:
import random
import numpy as np
def calculate_distance(point1, point2):
    return np.linalg.norm(point1-point2)

def select_points(points, num_points, min_distance):
    selected_points = []
    
    while len(selected_points) < num_points:
        point = random.choice(points)
        if all(calculate_distance(point, selected_point) >= min_distance for selected_point in selected_points):
            selected_points.append(point)
    
    return selected_points

In [8]:
from networkx.algorithms import isomorphism
from ase.neighborlist import natural_cutoffs, NeighborList
import networkx as nx
def is_unique(graph,unique_graphs):
    if unique_graphs==[]:
         return True
    for i,unique_graph in enumerate(unique_graphs):
        GM = isomorphism.GraphMatcher(graph, unique_graph, node_match=isomorphism.categorical_node_match('symbol', ''))
        if GM.is_isomorphic():
            return False
    return True

def get_graph(atoms):
    cutoffs = natural_cutoffs(atoms,mult=1.15)
    nl = NeighborList(cutoffs, self_interaction=False, bothways=True)
    nl.update(atoms)
    matrix = nl.get_connectivity_matrix(sparse=False)
    G = nx.Graph()
    symbols = atoms.symbols                               
    G.add_nodes_from([(i, {'symbol': symbols[i]}) 
                          for i in range(len(symbols))])
    rows, cols = np.where(matrix == 1)
    edges = zip(rows.tolist(), cols.tolist())
    G.add_edges_from(edges)
    graph = wl(G)
    return graph

def wl(graph):
        node_symbols = nx.get_node_attributes(graph, 'symbol')
        num_iterations = 3
        for _ in range(num_iterations):
            new_symbols = {}
            for node in graph.nodes():
                symbol = node_symbols[node]
                neighbor_symbols = [node_symbols[neighbor] for neighbor in graph.neighbors(node)]
                combined_symbol = symbol + ''.join(sorted(neighbor_symbols))
                new_symbols[node] = combined_symbol
            node_symbols = new_symbols

        new_graph = nx.Graph()
        for node, symbol in node_symbols.items():
            new_graph.add_node(node, symbol=symbol)

        return new_graph

In [ ]:
from ase import Atoms
sites = inside_sites + surf_sites
num_ad = 3
num_configuration = 10
symbol = 'H'
configurations = []
unique_graphs = []
for i in range(num_configuration):
    structure = atoms.copy()
    points = select_points(sites,num_ad,1.5)
    for p in points:
        ad = Atoms(symbols=symbol,positions=[p])
        structure += ad
    graph = get_graph(structure)
    if is_unique(graph,unique_graphs) == True:
        unique_graphs.append(graph)
    configurations.append(structure)
print(configurations)